# Dites-le avec des fleurs !

## Librairies utiles

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
# SeaBorn : librairie de graphiques avancés
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

## Le dataset des iris

Le dataset des iris est prédéfini dans Seaborn :

In [ ]:
iris = pd.read_csv("../input/iris.csv")

On peut afficher les 10 premières lignes du dataset :

In [ ]:
iris.head(10)

On a les informations suivantes :
- longueur du sépale (en cm)
- largeur du sépale
- longueur du pétale
- largeur du pétale
- espèce : Virginica, Setosa ou Versicolor

<img src="http://python.astrotech.io/_images/iris-flowers.png">

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQM3aH4Q3AplfE1MR3ROAp9Ok35fafmNT59ddXkdEvNdMkT8X6E">

## Données

Pour accéder à une colonne avec une notation pointée ou entre crochets :

In [ ]:
iris.species
iris['species']

La méthode *shape* permet de vérifier qu'on a 150 lignes et 5 colonnes

In [ ]:
iris.shape

On peut afficher le type de chaque colonne avec la méthode *info()*

In [ ]:
iris.info()

*describe()* permet d'afficher des statistiques sur le dataset :

In [ ]:
iris.describe()

On peut compter le nombre d'éléments de chaque espèce :

In [ ]:
iris.species.value_counts()

Avec une expression booléenne, on peut sélectionner les lignes correspondant à l'espèce Setosa dont les sépales sont plus longs que 5cm :

In [ ]:
iris[(iris.species=='setosa') & (iris.sepal_length>5)]

On peut aussi prédéfinir des booléens :

In [ ]:
setosa = iris.species=='setosa'
virginica = iris.species=='virginica'
versicolor = iris.species=='versicolor'
grand_sepale = iris.sepal_length>5

et les utiliser pour sélectionner des lignes :

In [ ]:
iris[setosa & grand_sepale]

** Pour plus de détails sur Pandas et la manipulation de Dataframes :  
https://github.com/jhroy/tuto-pandas/blob/master/tutoriel.ipynb  
https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python  **

## Visualisations

Pour tracer un histogramme, il faut définir le nombre de *bins* (barres de l'histogramme)

In [ ]:
plt.hist(iris.sepal_length, color="green", bins=100)

Ou seulement pour l'espèce Setosa :

In [ ]:
plt.hist(iris[setosa].sepal_length, bins=100)

Pour superposer plusieurs histogrammes, on peut définir une transparence avec le paramètre *alpha*

In [ ]:
plt.figure(figsize=(12,6))
plt.hist(iris[setosa].petal_length, color='blue', bins=50, alpha=0.3)
plt.hist(iris[versicolor].petal_length, color='red', bins=50, alpha=0.3)
plt.hist(iris[virginica].petal_length, color='green', bins=50, alpha=0.3)

*seaborn* (en abrégé *sns*) permet de réaliser des graphiques facilement :

In [ ]:
sns.catplot("petal_length", data=iris, hue="species", kind="count", height=8)

On peut également tracer estimation de la densité de probabilité d'une variable :

In [ ]:
sns.kdeplot(iris[setosa].sepal_length)

Ou en superposant distribution et histogramme :

In [ ]:
sns.distplot(iris[setosa].sepal_length)

*jointplot* permet de visualiser dans un plan les distributions d'un couple de paramètres :

In [ ]:
sns.jointplot("petal_length", "petal_width", iris, kind='kde');

Les **diagrammes en boîte** (ou **boîtes à moustaches** ou **box plot**) résument quelques caractéristiques de position du caractère étudié (médiane, quartiles, minimum, maximum ou déciles). Ce diagramme est utilisé principalement pour comparer un même caractère dans deux populations de tailles différentes. Il s'agit de tracer un rectangle allant du premier quartile au troisième quartile et coupé par la médiane. On ajoute alors des segments aux extrémités menant jusqu'aux valeurs extrêmes.  
Par exemple pour la répartion des espèces selon la longueur du sépale :

In [ ]:
sns.boxplot(x="species", y="sepal_length", data=iris)

Les **violins plots** sont similaires aux box plots, excepté qu’ils permettent de montrer la courbe de densité de probabilité des différentes valeurs. Typiquement, les violins plots présentent un marqueur pour la médiane des données et l’écart interquartile, comme dans un box plot standard.

In [ ]:
sns.violinplot(x="species", y="petal_length", data=iris)

*FacetGrid* permet de superposer des graphiques selon une ou plusieurs caractéristiques. On crée une structure avec *FacetGrid*, et on trace ensuite les graphiques avec *map*

In [ ]:
fig = sns.FacetGrid(iris, hue="species", aspect=3, palette="autumn") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "petal_width", shade=True)
fig.add_legend()

On veut tracer un nuage de points selon la longueur du pétale et la longueur du pétale, en différentiant les espèces :

In [ ]:
sns.lmplot(x="sepal_length", y="petal_length", data=iris, fit_reg=False, hue='species')

*pairplot* affiche les nuages de points associés à tous les couples de paramètres (attention, le temps de calcul peut être long) :

In [ ]:
sns.pairplot(iris[iris.species!='setosa'], hue="species")

## Machine learning

La plupart des algorithmes ont besoin de données numériques, et n'acceptent pas les chaînes de caractères. On va "mapper" les noms d'espèces vers des nombres, en créant une nouvelle colonne 'classe' :

In [ ]:
iris['classe'] = iris.species.map({"setosa":0, "versicolor":1, "virginica":2})

In [ ]:
iris.head()

On utilise *drop* pour supprimer la colone *species* (*axis=1* indique qu'on opère sur les colonnes et non sur les lignes):

In [ ]:
iris = iris.drop(['species'], axis=1)

In [ ]:
iris.head()

On sépare le dataset en deux parties :
- un ensemble d'apprentissage (entre 70% et 90% des données), qui va permettre d'entraîner le modèle
- un ensemble de test (entre 10% et 30% des données), qui va permettre d'estimer la pertinence de la prédiction

In [ ]:
data_train = iris.sample(frac=0.8)          # 80% des données avec frac=0.8
data_test = iris.drop(data_train.index)     # le reste des données pour le test

On sépare les données d'apprentissage (*X_train*) et la cible (*y_train*, la colonnes des données *classe*)

In [ ]:
X_train = data_train.drop(['classe'], axis=1)
y_train = data_train.classe
X_test = data_test.drop(['classe'], axis=1)
y_test = data_test.classe

## Régression logistique

On veut prédire une variable aléatoire $Y$ à partir d'un vecteur de variables explicatives $X=(X_1,...,X_n)$
On 


La fonction logistique $\frac{e^{x}}{1+e^{x}}$ varie entre $-\infty$ et $+\infty$ pour $x$ variant entre $0$ et $1$.  
Elle est souvent utilisée pour "mapper" une probabilité et un espace réel

In [ ]:
plt.figure(figsize=(9,9))

logistique = lambda x: np.exp(x)/(1+np.exp(x))   

x_range = np.linspace(-10,10,50)       
y_values = logistique(x_range)

plt.plot(x_range, y_values, color="red")

La régression logistique consiste à trouver une fonction linéaire C(X) qui permette d'estimer la probabilité de $Y=1$ connaissant $X$ :
$$p(Y=1|X) = \frac{e^{C(X)}}{1+e^{C(X)}}$$

Autrement dit, cela revient à trouver une séparation linéaire des caractéristiques qui minimise un critère d'erreur.

Pour plus de détails, cf par exemple :  
http://eric.univ-lyon2.fr/~ricco/cours/cours/pratique_regression_logistique.pdf

On veut maintenant prédire l'espèce à partir de toutes les caractéristiques, et évaluer la qualité de cette prédiction en utilisant la régression logistique définie dans la librairie *sklearn* :

In [ ]:
from sklearn.linear_model import LogisticRegression

On entraîne le modèle de régression logistique avec *fit* :

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

On peut prédire les valeurs sur l'ensemble de test avec le modèle entraîné :

In [ ]:
y_lr = lr.predict(X_test)

In [ ]:
X_test.shape

In [ ]:
print(np.array(y_test))
print(y_lr)

## Score et matrice de confusion

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

La mesure de pertinence compte le nombre de fois où l'algorithme a fait une bonne prédiction (en pourcentage) :

In [ ]:
lr_score = accuracy_score(y_test, y_lr)
print(lr_score)

Une mesure plus fine consiste à compter le nombre de **faux positif** (valeur prédite 1 et réelle 0) et de **vrai négatif** (valeur prédite 0 et réelle 1). On utilise une **matrice de confusion** :

In [ ]:
# Matrice de confusion
cm = confusion_matrix(y_test, y_lr)
print(cm)

<img src="https://i.stack.imgur.com/gKyb9.png">

## Arbres de décision

Un arbre de décision permet de faire à chaque étape un choix entre deux possibilités, pour arriver à une réponse sur les feuilles (cf. Akinator)  
<img src="https://scontent-cdg2-1.xx.fbcdn.net/v/t1.0-9/580017_474689062557542_1211572618_n.jpg?_nc_cat=100&oh=d87e6f9628499f5c872d0a375ab5c477&oe=5C28C2B9">

Pour construire un arbre de décision à partir d'un ensemble d'apprentissage, on va choisir une variable qui sépare l'ensemble en deux parties les plus distinctes en fonction d'un critère. Sur les iris par exemple, on peut utiliser la largeur du pétale pour séparer l'espèce Setosa des autres.

L'indice *GINI* mesure avec quelle fréquence un élément aléatoire de l'ensemble serait mal classé si son étiquette était sélectionnée aléatoirement depuis la distribution des étiquettes dans le sous-ensemble.

In [ ]:
from sklearn import tree
dtc = tree.DecisionTreeClassifier()
dtc.fit(X_train,y_train)
y_dtc = dtc.predict(X_test)
print(accuracy_score(y_test, y_dtc))

L'arbre de décision :
<img src="http://cedric.cnam.fr/vertigo/Cours/ml2/_images/tparbresiris.svg">

On peut modifier certains paramètres :  Le paramètre *max_depth* est un seuil sur la profondeur maximale de l’arbre. Le paramètre *min_samples_leaf* donne le nombre minimal d’échantillons dans un noeud feuille.

In [ ]:
dtc1 = tree.DecisionTreeClassifier(max_depth = 3, min_samples_leaf = 20)

On obtient un arbre un peu différent :
<img src="http://cedric.cnam.fr/vertigo/Cours/ml2/_images/tparbresiris1.svg">

In [ ]:
dtc1.fit(X_train,y_train)
y_dtc1 = dtc1.predict(X_test)
print(accuracy_score(y_test, y_dtc1))

Pour plus de détails sur les arbres de décision :  
https://zestedesavoir.com/tutoriels/962/les-arbres-de-decisions/comprendre-le-concept/#1-les-origines  
http://cedric.cnam.fr/vertigo/Cours/ml2/tpArbresDecision.html  
http://perso.mines-paristech.fr/fabien.moutarde/ES_MachineLearning/Slides/coursFM_AD-RF.pdf  

## Random forests

<img src="https://infinitescript.com/wordpress/wp-content/uploads/2016/08/Random-Forest-Example.jpg">

cf par exemple :  
https://fr.wikipedia.org/wiki/For%C3%AAt_d%27arbres_d%C3%A9cisionnels  
https://www.biostars.org/p/86981/  
https://infinitescript.com/2016/08/random-forest/

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
rf_score = accuracy_score(y_test, y_rf)
print(rf_score)

In [ ]:
cm = confusion_matrix(y_test, y_rf)
print(cm)

### Importance des caractéristiques

L'attribut *feature_importances_* renvoie un tableau du poids de chaque caractéristique dans la décision :

In [ ]:
importances = rf.feature_importances_
print(importances)
indices = np.argsort(importances)
print(indices)

On peut visualiser ces degrés d'importance avec un graphique à barres par exemple :

In [ ]:
plt.figure(figsize=(8,5))
plt.barh(range(len(indices)), importances[indices], color='r', align='center')
plt.yticks(range(len(indices)), iris.columns[indices])
plt.title('Importance des caracteristiques')

## Support Vector Machines

cf par exemple :  
https://medium.com/machine-learning-101/chapter-2-svm-support-vector-machine-theory-f0812effc72  
http://scikit-learn.org/stable/modules/svm.html  
http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html  
https://www.svm-tutorial.com/  
http://www.grappa.univ-lille3.fr/~ppreux/papiers/man.pdf
<img src = "http://scikit-learn.org/stable/_images/sphx_glr_plot_iris_001.png">

In [ ]:
from sklearn import svm
svc = svm.SVC()
svc.fit(X_train, y_train)
y_svc = svc.predict(X_test)

In [ ]:
svc_score = accuracy_score(y_test, y_svc)
print(svc_score)

## Clustering

Les problèmes de *segmentation* sont assez courants : par exemple, on souhaite répartir des clients en plusieurs catégories, pour différencier des envois publicitaires.  
On a un problème *non supervisé* : on suppose qu'on ne connaît pas la catégorie des données.  
La méthode la plus usuelle est celle des *k plus proches voisins*. On veut décomposer les données en *k* classes. On chosit aléatoirement *k* points dans l'espace des données, et affecte chaque donnée au groupe *i* si cette donnée est plus proche du point *i*. On recalcule alors les *k* barycentres de chaque groupe obtenu, et on recommence tant que les barycentres changent.  
cf par exemple :  
https://fr.wikipedia.org/wiki/Recherche_des_plus_proches_voisins  
http://www.grappa.univ-lille3.fr/polys/fouille/sortie005.html  
Il existe de nombreuses autres méthodes de clustering :  
http://scikit-learn.org/stable/modules/clustering.html  
https://hdbscan.readthedocs.io/en/latest/comparing_clustering_algorithms.html

In [ ]:
from sklearn.cluster import KMeans

On "oublie" l'espèce des iris :

In [ ]:
X_iris = iris.drop(['classe'], axis=1)

On utilise donc la méthode des k plus proches voisins (http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) :

In [ ]:
km = KMeans(n_clusters=4)

In [ ]:
km.fit(X_iris)

In [ ]:
iris_labels = km.predict(X_iris)

On peut vérifier qu'on retrouve (globalement) la décomposition en espèces (évidemment les numéros ne correspondent pas) :

In [ ]:
iris_labels, np.array(iris.classe)

On peut visualiser les clusters obtenus (ici on choisit en x la longueur de pétale, en y la largeur de pétale et en diamètre du point la longueur de sépale) :

In [ ]:
plt.scatter(X_iris.petal_length, X_iris.petal_width, c = iris_labels, cmap="Set3", s=(iris.sepal_length**2)*5)

Et on peut comparer aux espèces (les couleurs ne correspondent pas) :

In [ ]:
plt.scatter(X_iris.petal_length, X_iris.petal_width, c = iris.classe, cmap="Set3", s=(iris.sepal_length**2)*5)

On voit qu'on a bien retrouvé l'espèce Setosa, mais que l'on confond plus fréquemment Versicolor et Virginica

La **classification ascendante hiérarchique** (CAH) est une méthode de classification itérative dont le principe est simple.  
- On commence par calculer la dissimilarité entre les N objets.  
- Puis on regroupe les deux objets dont le regroupement minimise un critère d'agrégation donné, créant ainsi une classe comprenant ces deux objets.  
- On calcule ensuite la dissimilarité entre cette classe et les N-2 autres objets en utilisant le critère d'agrégation. Puis on regroupe les deux objets ou classes d'objets dont le regroupement minimise le critère d'agrégation.
On continue ainsi jusqu'à ce que tous les objets soient regroupés.

Ces regroupements successifs produisent un arbre binaire de classification (dendrogramme), dont la racine correspond à la classe regroupant l'ensemble des individus. Ce dendrogramme représente une hiérarchie de partitions. On peut alors choisir une partition en tronquant l'arbre à un niveau donné, le niveau dépendant soit des contraintes de l'utilisateur (l'utilisateur sait combien de classes il veut obtenir), soit de critères plus objectifs.  
https://www.xlstat.com/fr/solutions/fonctionnalites/classification-ascendante-hierarchique-cah  
https://perso.univ-rennes1.fr/valerie.monbet/ExposesM2/2013/Classification2.pdf  
https://fr.wikipedia.org/wiki/Regroupement_hi%C3%A9rarchique  

La méthode *clustermap* de seaborn parmet de visualiser un dendrogramme en 2 dimensions. Ici on utilise *rowcolors* pour visualiser la classe (espèce), qu'on associe à une couleur :

In [ ]:
sns.clustermap(X_iris, cmap="coolwarm", row_colors=iris.classe.map({0:"r",1:"g",2:"b"}), figsize=(12,12))

Qu'observez-vous ?

## MLPClassifier

In [ ]:
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
est = MLPClassifier(hidden_layer_sizes = (20,10))
est.max_iter = 2000
est.fit(X_train, y_train)
y_mlp = est.predict(X_test)
print(metrics.confusion_matrix(y_mlp, y_test))
print(metrics.classification_report(y_mlp, y_test))